# A/B Testing

## Background

A/B testing typically measures

- Number of clicks
- Number of purchases
- Duration of seeing web apges
- Number of pages visited
- Whether a particular page is visited

## Hypothesis test

### z test

### t test

### Type I error

### Type II error

## Random permutation test

Randomization test, resampling permutation procedure.

When samples don't follow a normal distribution

## Multiple A/B tests

Yes, we can. But we cannot purely run the A/B tests multiple times. We need to consider the following.

### Bonferroni correction (adjustment)

Divide significance level $\alpha$ by the number of multiple test $n$, so $\frac{\alpha}{n}$

### F-test

xxx

### Multi-arm bandit

xxx

## Compute sample size

We can compute sample size if we have **effect size**, **power**, and **significance level**.

The **effect size** is the amount of change we hope to have with a new feature. For example, with a new feature, if we want the click rate to increase by 10%, the 10% is the effect size.

The **power** is the probability of detecting the effect size in A/B testing.

The **significance level** is the probability of incorrectly saying we have the effect when the effect actually doesn't exist.

## Experimental design

### Multiple features

Button color and button location

### Interpret A/B testing result

new UI effect

### Apply A/B testing to price

### Randomization

Random bucketing

### Non-normal distribution

Mann-whitney, etc.

## Terminology

- Churn rate
  - The rate at which a customer stops doing business with a company, or stops subscribing to a service.
- Researcher bias
  - Selecting a test statistic after an experiment, but the test statistic needs to be established beforehand. 

## Reference

- O'REILLY Practical Statistics for Data Scientist, 50+ Essential Concepts Using R and Python
- [How to A/B Test Your Pricing (And Why It Might Be a Bad Idea)](https://blog.hubspot.com/marketing/price-testing)
- [Common Mistakes During A/B Testing](https://towardsdatascience.com/common-mistakes-during-a-b-testing-bdb9eefdc7f0)